In [ ]:
import pandas as pd

In [111]:
beta = pd.read_csv('beta_activity_class.csv') #Clean CSV file with beta secretase smiles and activity

In [112]:
beta

,Ligand SMILES,activity_class
0,CCSc1cnc(cn1)C(=O)Nc1cccc(c1)[C@]1(C)CCSC(N)=N1,moderately_active
1,C[C@]1(CCSC(N)=N1)c1cc(NC(=O)CCc2ccc(O)c(O)c2)...,moderately_active
2,COc1nc(nc(C)c1F)N1C[C@H]2C(=O)N(C)C(=N)N[C@]2(...,moderately_active
3,COc1nc(nc(C)c1F)N1C[C@H]2C(=O)N(C)C(=N)N[C@]2(...,moderately_active
4,COc1nc(nc(C)c1F)N1C[C@H]2C(=O)N(C)C(=N)N[C@]2(...,moderately_active
...,...,...
15992,FC(F)(F)c1ccc(N\N=C\c2coc3cc4oc(cc4cc3c2=O)-c2...,inactive
15993,COc1ccc(cc1)-c1cc2cc3c(cc2o1)occ(\C=N\Nc1ccc(c...,inactive
15994,COc1ccc(CN(CCN(C)CCN)c2ccccn2)cc1,inactive
15995,Fc1cccc(c1)-c1cc2c(ccc3c2occ(\C=N\Nc2ccc(cc2)C...,inactive


In [ ]:
#Create morgan fingerprints for compound smiles using RDKit (refer back to exploratory_DA)


In [ ]:
#transform the activty_class into integer labels for training

In [ ]:
#Train three non-neural net models for classification 

In [ ]:
#Neural net model for classification (you can refer to https://psrivasin.medium.com/multiclass-classification-using-tensorflow-850ee138d0ca for a guide)
#Input layer from morgan fingerprints
#hidden layer > hyperparameter tune 
#hyperparameter to tune: number of hidden layers (1 to 4), number of units in each layer (32-128), learning rate ( 1e-05 to 5e-05)
#dropout layer > tune dropout rate (0.1 to 0.3)
#Output 1 dense layer with 3 units, softmax activation